# Scrape Data From Monster For All States

In [1]:
import pandas as pd # For Data Storage
import requests # For website connection
from bs4 import BeautifulSoup # For HTML parsing
import time # For sleep 
import re # Regular expressions for removing non-ascii terms
from itertools import groupby # For removing duplicates from lists
import math # Need ceiling expression
from nltk.corpus import stopwords # For filtering out words like 'is', 'the', 'of'
stop_words = set(stopwords.words("english")) #initialize stopwords    
import matplotlib.pyplot as plt
import random
import datetime

# Initialize Empty Lists (For Dictionary)

In [2]:
#Initialize Empty State Set
state = []
job_title = []

#Initialize Empty Skills List
r = []
python = []
java = []
cpp = []
csharp = [] 
c = [] 
ruby = []
perl = []
sas = [] 
spss = []
matlab = []
javascript = []
scala = []
excel = []
vba = []
tableau = []
d3 = []
hadoop = []
mapreduce = []
spark = []
pig = []
hive = []
shark = []
oozie = []
zookeeper = []
flume = []
mahout = []
sql = [] 
mysql = []
tsql = []
oracle = []
postgresql = []
nosql =[]
sybase = []
rethinkdb = []
memcacheddb = []
couchdb = []
hbase = []
cassandra = []
mongodb = []
db2 = []
git = []
tensorflow = []
keras = []

skills = ['r',
'python',
'java',
'c++',
'c#',
'c',
'ruby',
'perl',
'sas',
'spss',
'matlab',
'javascript',
'scala', 
'excel',
'vba',
'tableau',
'd3',
'hadoop',
'mapreduce',
'spark',
'pig',
'hive',
'shark',
'oozie',
'zookeeper',
'flume',
'mahout',
'sql',
'tsql',
'mysql',
'oracle',
'postgresql',
'nosql',
'sybase',
'rethinkdb',
'memcacheddb',
'couchdb',
'hbase',
'cassandra',
'mongodb',
'db2',
'git',
'tensorflow',
'keras'
]

# Initialize Dictionary

In [3]:
dictionary = {
    'job_title':job_title,
    'state':state,
    'r':r,
    'python':python,
    'java':java,
    'c++':cpp,
    'c#':csharp,
    'c':c,
    'ruby':ruby,
    'perl':perl,
    'sas':sas,
    'spss':spss,
    'matlab':matlab,
    'javascript':javascript,
    'scala':scala,
    'excel':excel,
    'vba':vba,
    'tableau':tableau,
    'd3':d3,
    'hadoop':hadoop,
    'mapreduce':mapreduce,
    'spark':spark,
    'pig':pig,
    'hive':hive,
    'shark':shark,
    'oozie':oozie,
    'zookeeper':zookeeper,
    'flume':flume,
    'mahout':mahout,
    'sql':sql, 
    'tsql':tsql,
    'mysql':mysql,
    'oracle':oracle,
    'postgresql':postgresql,
    'nosql':nosql,
    'sybase':sybase,
    'rethinkdb':rethinkdb,
    'memcacheddb':memcacheddb,
    'couchdb':couchdb,
    'hbase':hbase,
    'cassandra':cassandra,
    'mongodb':mongodb,
    'db2':db2,
    'git':git,
    'tensorflow':tensorflow,
    'keras':keras}

# Define Func: Add A Row of Data To Dictionary (For a Job Description)

In [4]:
def addValsToDict(url_state,url_job_title):
    #Append job_title
    dictionary['job_title'].append(url_job_title)
    
    #Append state
    dictionary['state'].append(url_state)

    #Check if the each skill is in the wrangled text
    for skill in skills:
        if (skill in text_final) == True:
            dictionary[skill].append(1)
        else:
            dictionary[skill].append(0)

# Define States List

In [5]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

## import proxys

#Get top 10 proxies from http://free-proxy.cz/en/proxylist/country/US/https/ping/level1
https_proxys = ['34.192.220.22:808',
              '206.189.112.106:3128',
              '209.97.191.169:3128',
              '157.230.34.190:1111',
              '157.230.45.121:1111',
              '23.108.64.65:8118',
              '157.230.33.37:1111',
              '52.128.60.130:50692',
              '12.218.209.130:53281',
              '38.134.10.106:53281'
               ]

In [6]:
# Get proxies from filter
https_proxys = list(pd.read_csv('filtered_https_proxys.txt',header=None)[0])

## import headers

In [7]:
user_agents = pd.read_csv('headers.txt',delimiter = '\t',header=None).T[0]

In [8]:
user_agents = user_agents[0:-1] # Get rid of last element (it is nan)

In [9]:
user_agents = list(user_agents)

# Create select a random proxy and header funcitons

In [10]:
def randIndex(lengthList):
    index = random.randint(0,lengthList-1)
    return index

def randProxy():
    index = randIndex(len(https_proxys))
    return https_proxys[index]
    
def randHeader():
    index = randIndex(len(user_agents))
    return user_agents[index]

# Initialize Headers Dictionary

In [11]:
header = {'User-Agent':randHeader()}

# Initialize Proxy Dictionary

In [12]:
proxy = {'https':randProxy()}

## For Each State:

- ## Get the urls that lists up to 250 job openings

In [13]:
# Set the general search and use lowercases and dashes ('-') in place of capitals and spaces 
# i.e a search for the term Professional Baseball Player would be search_term = 'professional-baseball-player'
search_term = 'software-engineer'

In [14]:
count = 0

# Initialize Empty URL Dictionary
final_state_url_dict = {} 

while len(final_state_url_dict.keys()) < 50:
    for url_state in states:
        try:
            if url_state not in final_state_url_dict.keys():
                time.sleep(1) # sleep 1 second

                # Assign random proxy
                proxy['https'] = randProxy()

                # Assign random header
                header['User-Agent'] = randHeader()

                count += 1
                print("Number of states left to scrape: "+str(50-count))

                url = 'https://www.monster.com/jobs/search/?q='+search_term+'&where='+url_state+'&stpage=1'        

                response = requests.get(url, proxies=proxy, headers=header, timeout=5) #Try to get the html code

                soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

                time.sleep(1) #pause code for 1 sec
                
                # Sometimes there are no job openings and the float conversion throws an error
                # When this float conversion error occurs, we set the total total job openings equal to zero
                try: 
                    #Get the total number of listed jobs 
                    totalJobOpenings = float(re.sub('[^0-9]','',soup.html.body.find_all('h2')[0].text.strip()))
                
                except Exception,e: 
                    print str(e)
                    totalJobOpenings = 0
                    print url_state+' has no jobs.'
                    
                #Get the total number of pages (up to 10) and a max of 25 per page
                totalPages = int(min(math.ceil(totalJobOpenings/25),10))

                #Store the Final URL in a dictionary each state
                final_state_url_dict[url_state] = 'https://www.monster.com/jobs/search/?q='+search_term+'&where='+url_state+'&stpage=1&page='+str(totalPages)


        except Exception,e: 
            count -= 1
            print str(e)

Number of states left to scrape: 49
Number of states left to scrape: 48
Number of states left to scrape: 47
Number of states left to scrape: 46
Number of states left to scrape: 45
Number of states left to scrape: 44
Number of states left to scrape: 43
('Connection aborted.', BadStatusLine("''",))
Number of states left to scrape: 43
HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=software-engineer&where=DE&stpage=1 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of states left to scrape: 43
Number of states left to scrape: 42
Number of states left to scrape: 41
HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=software-engineer&where=HI&stpage=1 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1f26fd50>: Failed to establish a new connection: [Errno 61] Connection refused',

Number of states left to scrape: 4
Number of states left to scrape: 3
Number of states left to scrape: 2
Number of states left to scrape: 1
Number of states left to scrape: 0
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=5)
Number of states left to scrape: 0


- ## Get the list of job post URLs -> For each URL: check if each skill exists and store data in dictionary

In [15]:
# End the loop when all job state urls have been processed
while len(final_state_url_dict.keys()) > 0:
    for state in final_state_url_dict.keys():
        try:
            print("Number of states left to scrape: "+str(len(final_state_url_dict.keys())))
            time.sleep(1) # sleep 1 second

            # Assign random proxy
            proxy['https'] = randProxy()

            # Assign random header
            header['User-Agent'] = randHeader()
            
            #Access the job listing page for a state
            response = requests.get(final_state_url_dict[state],proxies=proxy,headers=header,timeout=3) #Get the html code
            soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

            links = []
            job_titles = {} 
            
            # If the last two letters are not '=0' then there are actually job posts to scrape
            if final_state_url_dict[state][-2:] != '=0':
                # Access every URL within the page and store only urls for job openings
                for link in soup.find_all('a'):
                    if link.get('href')[0:21] == 'https://job-openings.': ## a job opening always begins with 'https://job-openings.monster.com'
                        links.append(link.get('href'))
                        job_titles[link.get('href')] = link.contents[0].strip('\n').strip('\r')
            else: 
                #delete states job search url and end the loop early
                del final_state_url_dict[state]
                print state+'has no jobs'
                break
           
            
                
            # Access each job posting and store it's html code into a beautiful soup object
            # End the loop when all job title urls have been processed
            while len(job_titles.keys()) > 0: 
                for url in job_titles.keys():
                    
                    try:
                        # Assign random proxy
                        proxy['https'] = randProxy()

                        # Assign random header
                        header['User-Agent'] = randHeader()

                      #  count2 += 1
                        print("Number of jobs left to scrape: "+str(len(job_titles.keys())))

                        response = requests.get(url,proxies=proxy,headers=header,timeout=3) #Get the html code
                        soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 
                        #Get and clean the text from the job posting
                        text = soup.body.find_all('script')[1]
                        # Convert to string -> Convert to lower case 
                        text_wrangled = str(text).lower()
                        # replace (all non ascii characters,'.' ,'+' , and '3') with ','
                        text_wrangled = re.sub("[^a-zA-Z.+#234]",',',text_wrangled)
                        # split string into a list separated by ','
                        text_wrangled = text_wrangled.split(',')
                        # remove all consecutive duplicates 
                        text_wrangled = groupby(text_wrangled)
                        text_wrangled = [x[0] for x in text_wrangled]
                        # Remove the stopwords and empty spaces
                        text_final = [x for x in text_wrangled if not x in stop_words and not x =='']
                        #Add the values to the Dictionary
                        addValsToDict(state,job_titles[url])

                        #delete the job post link when finished
                        del job_titles[url]
                        
                    except Exception,e: 
                        print str(e)
                        
            #delete states job search url
            del final_state_url_dict[state]
       
        except Exception,e: 
            print str(e)
            
            

Number of states left to scrape: 50
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=3)
Number of states left to scrape: 50
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=3)
Number of states left to scrape: 50
HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=software-engineer&where=WI&stpage=1&page=10 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of states left to scrape: 50
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=3)
Number of states left to scrape: 50
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=3)
Number of states left to scrape: 50
HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=software-engineer&where=FL&stpage=1&page=10 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed o

Number of jobs left to scrape: 109
Number of jobs left to scrape: 108
Number of jobs left to scrape: 107
Number of jobs left to scrape: 106
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /12y-geospatial-engineer-cheyenne-wy-us-army-national-guard/207258087 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1f4cc610>: Failed to establish a new connection: [Errno 61] Connection refused',)))
Number of jobs left to scrape: 106
Number of jobs left to scrape: 105
Number of jobs left to scrape: 104
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /cloud-services-senior-engineer-vmware-cheyenne-wy-us-green-house-data/cfdbec7b-d4d8-472c-9dae-f138584e498a (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a2006fb90>: Failed to establish a new connection

('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 89
Number of jobs left to scrape: 88
Number of jobs left to scrape: 87
Number of jobs left to scrape: 86
list index out of range
Number of jobs left to scrape: 86
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /account-executive-wyoming-wy-us-accruent/50f8bd20-400e-411a-97dd-014a43375238 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1f3ac990>, 'Connection to 206.12.89.89 timed out. (connect timeout=3)'))
Number of jobs left to scrape: 86
Number of jobs left to scrape: 85
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /armo-technician-evansville-wy-us-baker-hughes/76c23285-4454-4b53-9180-50b2dcfaf97a (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of jobs left to scrape: 85
Number of jobs left to scrape: 84
HTTPSConnectionPool(host='job-openi

Number of jobs left to scrape: 59
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /cdl-a-truck-drivers-hiring-immediately-5-000-sign-on-bon-huntley-wy-us-hvh-transportation-inc/dfbf333d-39d2-46a2-9920-1642e4f15020 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x109116750>: Failed to establish a new connection: [Errno 61] Connection refused',)))
Number of jobs left to scrape: 59
Number of jobs left to scrape: 58
Number of jobs left to scrape: 57
Number of jobs left to scrape: 56
Number of jobs left to scrape: 55
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /digital-marketing-specialist-thayne-wy-us-silver-star-broadband-fiber-circuits-phone-systems-security-control-systems-wireless-phone/9151336c-6ec2-4377-92fb-789ce3d82333 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of jobs lef

Number of jobs left to scrape: 18
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /digital-marketing-specialist-thayne-wy-us-silver-star-broadband-fiber-circuits-phone-systems-security-control-systems-wireless-phone/9151336c-6ec2-4377-92fb-789ce3d82333 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1f22fe10>: Failed to establish a new connection: [Errno 61] Connection refused',)))
Number of jobs left to scrape: 18
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /senior-windows-administrator-cheyenne-wy-us-dish-network/426f7949-1202-4641-9c4b-4295d438a1ee (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1f22ff50>: Failed to establish a new connection: [Errno 61] Connection refused',)))
Number of jobs left to scrape: 18
Number of jobs lef

...

('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 57
Number of jobs left to scrape: 56
Number of jobs left to scrape: 55
Number of jobs left to scrape: 54
Number of jobs left to scrape: 53
Number of jobs left to scrape: 52
Number of jobs left to scrape: 51
Number of jobs left to scrape: 50
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /software-robotics-engineer-3-factory-automation-wichita-ks-us-spirit-aerosystems/6e60d938-76cd-4c90-b811-4e170385e564 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1f39fa90>, 'Connection to 187.189.249.26 timed out. (connect timeout=3)'))
Number of jobs left to scrape: 50
Number of jobs left to scrape: 49
Number of jobs left to scrape: 48
Number of jobs left to scrape: 47
Number of jobs left to scrape: 46
Number of jobs left to scrape: 45
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /appl

HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /software-robotics-engineer-3-factory-automation-wichita-ks-us-spirit-aerosystems/6e60d938-76cd-4c90-b811-4e170385e564 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a203bd310>, 'Connection to 134.209.119.225 timed out. (connect timeout=3)'))
Number of jobs left to scrape: 9
Number of jobs left to scrape: 8
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /android-developer-overland-park-ks-us-levi-ray-shoup-inc/71096510-3bc3-4444-87d4-afe48bb8f8f1 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a20c48150>, 'Connection to 94.242.58.108 timed out. (connect timeout=3)'))
Number of jobs left to scrape: 8
Number of jobs left to scrape: 7
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /entry-mid-lelvel-net-developer-ii-overla

HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=software-engineer&where=AZ&stpage=1&page=10 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a20113850>: Failed to establish a new connection: [Errno 61] Connection refused',)))
Number of states left to scrape: 28
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=3)
Number of states left to scrape: 28
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=3)
Number of states left to scrape: 28
HTTPSConnectionPool(host='www.monster.com', port=443): Read timed out. (read timeout=3)
Number of states left to scrape: 28
HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=software-engineer&where=MA&stpage=1&page=10 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1

HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /java-j2ee-developer-hibernate-spring-omaha-ne-us-sonsoft-inc/5ad5e148-76d9-448e-b4f0-39daac950fe3 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of jobs left to scrape: 240
Number of jobs left to scrape: 239
Number of jobs left to scrape: 238
Number of jobs left to scrape: 237
Number of jobs left to scrape: 236
Number of jobs left to scrape: 235
Number of jobs left to scrape: 234
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /software-engineer-bellevue-ne-us-leidos/5b055c6c-a70b-4a64-a360-3c335848fd31 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of jobs left to scrape: 234
Number of jobs left to scrape: 233
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /software-developer-jboss-linux-unix-java-omaha-ne-omaha-ne-us-huff-consult

KeyboardInterrupt: 

# Make the DataFrame

In [16]:
data = pd.DataFrame(dictionary)

# Store to File

In [17]:
now = datetime.datetime.now()
date = str(now.month)+'-'+str(now.day)+'-'+str(now.year)
data.to_csv('scraped_data/scraped_data_'+search_term+'_'+date+'.txt',encoding='utf-8',index=None)

# Check the File 

In [18]:
df = pd.read_csv('scraped_data_'+search_term+'_'+date+'.txt')